In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing

In [2]:
data = pd.read_csv('1830151.csv')

In [3]:
df=data.copy()

In [4]:
df['clear_date']=pd.to_datetime(df['clear_date'])
df['posting_date']=pd.to_datetime(df['posting_date'])
df['due_in_date']=pd.to_datetime(df['due_in_date'],format='%Y%m%d', errors='ignore')
df['document_create_date']=pd.to_datetime(df['document_create_date'],format='%Y%m%d', errors='ignore')
df['document_create_date.1']=pd.to_datetime(df['document_create_date.1'],format='%Y%m%d', errors='ignore')
df['baseline_create_date']=pd.to_datetime(df['baseline_create_date'], format='%Y%m%d', errors='ignore')
df['buisness_year']=df['buisness_year'].astype(int)

In [5]:
df.drop(['document_create_date'],axis=1,inplace=True)
df=df.sort_values(by='document_create_date.1',ignore_index=True)
df=df.rename(columns={'document_create_date.1':'document_create_date'})

In [6]:
data_train=df.loc[data['clear_date'].notna()]
data_train=data_train.reset_index(drop=True)
data_null_cleardate=df.loc[data['clear_date'].isna()]
data_null_cleardate=data_null_cleardate.reset_index(drop=True)

In [7]:
df=data_train

In [8]:
df=(df.loc[df['document_create_date']<=df['posting_date']].loc[df['posting_date']<=df['due_in_date']].loc[df['baseline_create_date']<=df['due_in_date']].loc[df['baseline_create_date']<=df['clear_date']])

In [9]:
df_data=df.copy()

In [10]:
#df=df_data.copy()

In [11]:
df['due_duration']=(df['clear_date']-df['due_in_date']).dt.days

In [12]:
df['cust_delay_mean']=df['cust_number'].map(df.groupby('cust_number')['due_duration'].mean())

df['payment_time']=(df['due_in_date']-df['document_create_date']).dt.days

In [13]:
df=df.drop(['posting_id','area_business','isOpen','clear_date','document type'],axis=1)

In [14]:
#df.drop(['clear_date'],axis=1,inplace=True)

In [15]:
x_train, x_test_init, y_train, y_test_init = train_test_split(
    df.drop(labels=['due_duration'], axis=1),
    df['due_duration'],
    test_size=0.3,
    shuffle=False)
x_train.shape,  y_train.shape, x_test_init.shape, y_test_init.shape

((32073, 15), (32073,), (13746, 15), (13746,))

In [16]:
x_val, x_test = train_test_split(
    x_test_init,
    test_size=0.5,
    shuffle=False)
y_val, y_test = train_test_split(
y_test_init,
    test_size=0.5,
    shuffle=False)
x_val.shape, x_test.shape,y_val.shape, y_test.shape

((6873, 15), (6873, 15), (6873,), (6873,))

In [46]:
df=x_train.copy()

In [47]:
def encoding(df,a):
    df_obj=df#[['name_customer','cust_number','cust_payment_terms','business_code','invoice_currency']].copy()
    code_invoice_currency = {k: i for i, k in enumerate(a.invoice_currency.unique(), 1)}
    df_obj.loc[:, 'code_invoice_currency'] = df_obj.loc[:, 'invoice_currency'].map(code_invoice_currency)

    code_business_code = {k: i for i, k in enumerate(a.business_code.unique(), 1)}
    df_obj.loc[:, 'code_business_code'] = df_obj.loc[:, 'business_code'].map(code_business_code)

    code_cust_payment_terms = {k: i for i, k in enumerate(a.cust_payment_terms.unique(), 1)}
    df_obj.loc[:, 'code_cust_payment_terms'] = df_obj.loc[:, 'cust_payment_terms'].map(code_cust_payment_terms)

    code_cust_number = {k: i for i, k in enumerate(a.cust_number.unique(), 1)}
    df_obj.loc[:, 'code_cust_number'] = df_obj.loc[:, 'cust_number'].map(code_cust_number)

    code_name_customer = {k: i for i, k in enumerate(a.name_customer.unique(), 1)}
    df_obj.loc[:, 'code_name_customer'] = df_obj.loc[:, 'name_customer'].map(code_name_customer)
    
    #df['name_customer'],df['cust_number'],df['cust_payment_terms'],df['business_code'],df['invoice_currency']=df_obj.loc[:, 'code_name_customer'],df_obj.loc[:, 'code_cust_number'],df_obj.loc[:, 'code_cust_payment_terms'],df_obj.loc[:, 'code_business_code'],df_obj.loc[:, 'code_invoice_currency']
    return df

In [48]:
def dates_feature(df):
    df_dates=df
    df_dates['posting_date_month'],df_dates['posting_date_day'],df_dates['posting_date_year'],df_dates['posting_date_quarter'],df_dates['posting_date_dayofweek'],df_dates['posting_date_week']=df_dates['posting_date'].dt.month,df_dates['posting_date'].dt.day,df_dates['posting_date'].dt.year,df_dates['posting_date'].dt.quarter,df_dates['posting_date'].dt.dayofweek,df_dates['posting_date'].dt.isocalendar().week.astype(int)
    df_dates['due_in_date_month'],df_dates['due_in_date_day'],df_dates['due_in_date_year'],df_dates['due_in_date_quarter'],df_dates['due_in_date_dayofweek'],df_dates['due_in_date_week']=df_dates['due_in_date'].dt.month,df_dates['due_in_date'].dt.day,df_dates['due_in_date'].dt.year,df_dates['due_in_date'].dt.quarter,df_dates['due_in_date'].dt.dayofweek,df_dates['due_in_date'].dt.isocalendar().week.astype(int)
    df_dates['document_create_date_month'],df_dates['document_create_date_day'],df_dates['document_create_date_year'],df_dates['document_create_date_quarter'],df_dates['document_create_date_dayofweek'],df_dates['document_create_date_week']=df_dates['document_create_date'].dt.month,df_dates['document_create_date'].dt.day,df_dates['document_create_date'].dt.year,df_dates['document_create_date'].dt.quarter,df_dates['document_create_date'].dt.dayofweek,df_dates['document_create_date'].dt.isocalendar().week.astype(int)
    df_dates['baseline_create_date_month'],df_dates['baseline_create_date_day'],df_dates['baseline_create_date_year'],df_dates['baseline_create_date_quarter'],df_dates['baseline_create_date_dayofweek'],df_dates['baseline_create_date_week']=df_dates['baseline_create_date'].dt.month,df_dates['baseline_create_date'].dt.day,df_dates['baseline_create_date'].dt.year,df_dates['baseline_create_date'].dt.quarter,df_dates['baseline_create_date'].dt.dayofweek,df_dates['baseline_create_date'].dt.isocalendar().week.astype(int)
    df=df_dates
    #df=df.drop(['posting_date','document_create_date','due_in_date','baseline_create_date'],axis=1)
    return df

In [49]:
def feature_set(df,a):
    df=encoding(df,a)
    df=dates_feature(df)
    #df.drop(['posting_date','document_create_date','due_in_date','baseline_create_date'],axis=1,inplace=True)
    #print(df.head(3))
    #df.drop(['buisness_year','code_invoice_currency','code_business_code','due_in_date_year','document_create_date_year'],axis=1,inplace=True)
    #print(df.head(3))
    #df.drop(['doc_id', 'invoice_id'],axis=1,inplace=True)
    df.drop(['name_customer','cust_number','cust_payment_terms','business_code','invoice_currency'],axis=1,inplace=True)
    #print(df.head(3))
    df=df.drop(['posting_date','document_create_date','due_in_date','baseline_create_date'],axis=1)
    df=df.drop(['buisness_year',
     'code_invoice_currency',
     'code_business_code',
     'posting_date_year',
     'due_in_date_year',
     'document_create_date_year',
     'baseline_create_date_year'],axis=1)
    df=df.drop(['doc_id', 'invoice_id'],axis=1)
    return df

In [50]:
df=feature_set(df,x_train)

In [51]:
df.columns.value_counts().sum()

26

In [52]:
#df=feature_set(,x_train)

In [53]:
from pandas.api.types import is_numeric_dtype
def remove_outlier(df):
    low = .05
    high = .95
    quant_df = df.quantile([low, high])
    for name in list(df.columns):
        if is_numeric_dtype(df[name]):
            df = df[(df[name] > quant_df.loc[low, name]) & (df[name] < quant_df.loc[high, name])]
    return df
#remove_outlier(df).shape

In [54]:
df.columns

Index(['total_open_amount', 'cust_delay_mean', 'payment_time',
       'code_cust_payment_terms', 'code_cust_number', 'code_name_customer',
       'posting_date_month', 'posting_date_day', 'posting_date_quarter',
       'posting_date_dayofweek', 'posting_date_week', 'due_in_date_month',
       'due_in_date_day', 'due_in_date_quarter', 'due_in_date_dayofweek',
       'due_in_date_week', 'document_create_date_month',
       'document_create_date_day', 'document_create_date_quarter',
       'document_create_date_dayofweek', 'document_create_date_week',
       'baseline_create_date_month', 'baseline_create_date_day',
       'baseline_create_date_quarter', 'baseline_create_date_dayofweek',
       'baseline_create_date_week'],
      dtype='object')

In [55]:
Xtr=df

In [27]:
def iqr(data):
    Q1 = np.percentile(data, 25, interpolation = 'midpoint')  
    Q2 = np.percentile(data, 50, interpolation = 'midpoint')  
    Q3 = np.percentile(data, 75, interpolation = 'midpoint')  
    IQR = Q3 - Q1  
    low_lim = Q1 - 1.5 * IQR 
    up_lim = Q3 + 1.5 * IQR 
    outlier=[]
    for x in data: 
        if ((x> up_lim) or (x<low_lim)): 
             outlier.append(x) 
    print('outlier in the dataset is', len(outlier)) 
    return low_lim,up_lim

In [28]:
#poi=y_train
#low_lim,up_lim=iqr(poi)
#for i in df.columns:
 #   print(i)
  #  iqr(df[i])

In [29]:
def stan(a,n):
    if n==1:
        scaler = StandardScaler()
    elif n==2:
        scaler = preprocessing.QuantileTransformer(output_distribution='normal', random_state=0)
    return scaler

In [30]:
scaler=stan(df,2)

scaler.fit(df)

Xtr=scaler.transform(df)

In [121]:
Xtr.columns.value_counts().sum()

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [37]:
#Xtr=df.copy()

In [43]:
Xte=x_val.copy()

In [44]:
Xte=feature_set(Xte,x_train)

In [57]:
Xtr

,total_open_amount,cust_delay_mean,payment_time,code_cust_payment_terms,code_cust_number,code_name_customer,posting_date_month,posting_date_day,posting_date_quarter,posting_date_dayofweek,...,document_create_date_month,document_create_date_day,document_create_date_quarter,document_create_date_dayofweek,document_create_date_week,baseline_create_date_month,baseline_create_date_day,baseline_create_date_quarter,baseline_create_date_dayofweek,baseline_create_date_week
1,48419.67,0.541935,15,1,1,1,12,30,4,6,...,12,30,4,6,52,12,30,4,6,52
2,59427.93,9.142857,10,2,2,2,12,30,4,6,...,12,30,4,6,52,12,30,4,6,52
3,11309.19,2.368071,15,1,3,3,12,30,4,6,...,12,30,4,6,52,12,30,4,6,52
4,1248.12,-2.241145,15,3,4,4,12,30,4,6,...,12,30,4,6,52,12,30,4,6,52
5,1763.96,-2.241145,15,3,4,5,12,30,4,6,...,12,30,4,6,52,12,30,4,6,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32182,39618.64,2.391975,15,1,46,648,10,8,4,1,...,10,8,4,1,41,10,8,4,1,41
32183,62332.70,6.063830,15,1,82,1327,10,8,4,1,...,10,8,4,1,41,10,8,4,1,41
32184,19033.40,1.425743,15,1,12,937,10,8,4,1,...,10,8,4,1,41,10,8,4,1,41
32185,8018.58,-6.561039,15,1,33,107,10,8,4,1,...,10,8,4,1,41,10,8,4,1,41


In [45]:
Xte.columns

Index(['total_open_amount', 'cust_delay_mean', 'payment_time',
       'code_cust_payment_terms', 'code_cust_number', 'code_name_customer',
       'posting_date_month', 'posting_date_day', 'posting_date_quarter',
       'posting_date_dayofweek', 'posting_date_week', 'due_in_date_month',
       'due_in_date_day', 'due_in_date_quarter', 'due_in_date_dayofweek',
       'due_in_date_week', 'document_create_date_month',
       'document_create_date_day', 'document_create_date_quarter',
       'document_create_date_dayofweek', 'document_create_date_week',
       'baseline_create_date_month', 'baseline_create_date_day',
       'baseline_create_date_quarter', 'baseline_create_date_dayofweek',
       'baseline_create_date_week'],
      dtype='object')

In [33]:
Xte.columns.value_counts().sum()

26

In [34]:
#Xte=Xte.drop(['code_name_customer','due_in_date_week','document_create_date_week','code_cust_payment_terms','payment_time'],axis=1)

In [35]:
Xte['code_cust_payment_terms'].fillna(1,inplace=True)
Xte['code_cust_number'].fillna(4,inplace=True)
Xte['code_name_customer'].fillna(9,inplace=True)

In [36]:
#df=Xte
#df=df.drop(['posting_date','document_create_date','due_in_date','baseline_create_date'],axis=1)
#df=df.drop(['buisness_year',
 #    'code_invoice_currency',
  #   'code_business_code',
   #  'posting_date_year',
    # 'due_in_date_year',
     #'document_create_date_year'],axis=1)
#df=df.drop(['doc_id', 'invoice_id'],axis=1)

In [37]:
Xte.columns

Index(['total_open_amount', 'cust_delay_mean', 'payment_time',
       'code_cust_payment_terms', 'code_cust_number', 'code_name_customer',
       'posting_date_month', 'posting_date_day', 'posting_date_quarter',
       'posting_date_dayofweek', 'posting_date_week', 'due_in_date_month',
       'due_in_date_day', 'due_in_date_quarter', 'due_in_date_dayofweek',
       'due_in_date_week', 'document_create_date_month',
       'document_create_date_day', 'document_create_date_quarter',
       'document_create_date_dayofweek', 'document_create_date_week',
       'baseline_create_date_month', 'baseline_create_date_day',
       'baseline_create_date_quarter', 'baseline_create_date_dayofweek',
       'baseline_create_date_week'],
      dtype='object')

In [38]:
Xte=scaler.transform(Xte)

In [46]:
clf = RandomForestRegressor()
qwerty=clf.fit(Xtr, y_train)
predicted = clf.predict(Xte)
predicted1 = clf.predict(Xtr)

In [39]:
clf = RandomForestRegressor(n_estimators=35,max_depth=30,min_samples_leaf=10,min_samples_split=5)
qwerty=clf.fit(Xtr, y_train)
predicted = clf.predict(Xte)
predicted1 = clf.predict(Xtr)

In [40]:
print(qwerty.score(Xte, y_val))
qwerty.score(Xtr, y_train)

0.5880471696846724


0.6723821578108073

In [58]:
#Xte.columns

In [41]:
predicted = clf.predict(Xte)
print(mean_squared_error(y_val, predicted))
print(np.sqrt(mean_squared_error(y_val, predicted)))
print(r2_score(y_val, predicted))
print((abs((pd.Series(np.around(predicted)).astype(int)-y_val.reset_index(drop=True)))<4).value_counts())
mean_absolute_error(y_val, predicted)

20.57953088463652
4.536466784253635
0.5880471696846724
True     5490
False    1383
dtype: int64


2.49518729768402

In [228]:
predicted=pd.DataFrame(predicted)
predicted1=pd.DataFrame(predicted1)

In [193]:
clf = DecisionTreeRegressor(max_depth=27,min_samples_split=2,max_features=5)
qwerty=clf.fit(Xtr, y_train)
predicted = clf.predict(Xte)
print(qwerty.score(Xte, y_val))
print(qwerty.score(Xtr, y_train))

-12.910066124545677
0.965811134378724


In [195]:
predicted = clf.predict(Xte)
print(mean_squared_error(y_val, predicted))
print(r2_score(y_val, predicted))
print((abs((pd.Series(np.around(predicted)).astype(int)-y_val.reset_index(drop=True)))<4).value_counts())
mean_absolute_error(y_val, predicted)

694.8917797173658
-12.910066124545677
True     3488
False    3385
dtype: int64


13.369077828030775

In [184]:
classifier = DecisionTreeClassifier(max_depth=27,min_samples_split=2,max_features=5)
qwerty=classifier.fit(Xtr, y_train)
print(qwerty.score(Xte, y_val))

qwerty.score(Xtr, y_train)

0.24385275716572094


0.978767187353849

In [192]:
predicted = classifier.predict(Xte)
print(mean_squared_error(y_val, predicted))
print(r2_score(y_val, predicted))
print((abs((pd.Series(np.around(predicted)).astype(int)-y_val.reset_index(drop=True)))<4).value_counts())
mean_absolute_error(y_val, predicted)

110.05543430816238
-1.203045730681672
True     4159
False    2714
dtype: int64


5.034337261748872

In [179]:
classifier = RandomForestClassifier(n_estimators=75,max_depth=27,min_samples_split=5,max_features=10)
qwerty=classifier.fit(Xtr, y_train)
print(qwerty.score(Xte, y_val))

qwerty.score(Xtr, y_train)

0.43474465298996073


0.9261060705266112

In [181]:
predicted = classifier.predict(Xte)
print(mean_squared_error(y_val, predicted))
print(r2_score(y_val, predicted))
print((abs((pd.Series(np.around(predicted)).astype(int)-y_val.reset_index(drop=True)))<4).value_counts())
mean_absolute_error(y_val, predicted)

26.480139676996945
0.46993113943001497
True     5383
False    1490
dtype: int64


2.4228139095009458